In [2]:
res = ""

In [3]:
from safeds.data.image.containers import ImageList

rice_images, filenames = ImageList.from_files(res, return_filenames=True, load_percentage=0.4)

In [4]:
import re
from safeds.data.tabular.containers import Column

labels = Column(
    "label", 
    [re.search(r"(.*)[\\/](.*)[\\/](.*)\.", filepath).group(2) for filepath in filenames]
)
labels

label
str
"""Ipsala"""
"""Jasmine"""
"""Arborio"""
"""Jasmine"""
"""Basmati"""
…
"""Arborio"""
"""Karacadag"""
"""Basmati"""


In [11]:
from safeds.data.labeled.containers import ImageDataset

dataset = ImageDataset(rice_images, labels, batch_size=32, shuffle=True)

In [12]:
len(dataset)

30000

In [6]:
from safeds.ml.nn.layers import Convolutional2DLayer, MaxPooling2DLayer, FlattenLayer, ForwardLayer

layers = [
    Convolutional2DLayer(16, 5, padding=2),
    MaxPooling2DLayer(3, stride=3),
    Convolutional2DLayer(32, 5, padding=2),
    MaxPooling2DLayer(3, stride=3),
    Convolutional2DLayer(64, 5, padding=2),
    MaxPooling2DLayer(2, stride=2),
    Convolutional2DLayer(128, 5, padding=2),
    MaxPooling2DLayer(2, stride=2),
    FlattenLayer(),
    ForwardLayer(1024),
    ForwardLayer(256),
    ForwardLayer(len(labels.get_distinct_values())),
]

In [7]:
from safeds.ml.nn import NeuralNetworkClassifier
from safeds.ml.nn.converters import InputConversionImage, OutputConversionImageToColumn

cnn = NeuralNetworkClassifier(InputConversionImage(dataset.input_size), layers, OutputConversionImageToColumn())

In [14]:
def epoch_completion(epoch_number: int, epoch_loss: float):
    print(f"epoch [{epoch_number}]: {epoch_loss}")

In [16]:
trained_cnn = cnn.fit(dataset, 20, 32, callback_on_epoch_completion=epoch_completion)

epoch [1]: 1.6092273578969145
epoch [2]: 1.6086994515044857
epoch [3]: 1.6081099033609891
epoch [4]: 1.607268920331113
epoch [5]: 1.6058132883582288
epoch [6]: 1.6029890539295384
epoch [7]: 1.594318954293916
epoch [8]: 1.4976849730080888
epoch [9]: 0.644023995361984
epoch [10]: 0.247273467308375
epoch [11]: 0.15271162412075728
epoch [12]: 0.12474599635617724
epoch [13]: 0.10919117854475212
epoch [14]: 0.10531921253384732
epoch [15]: 0.10134941124434728
epoch [16]: 0.09989633529892823
epoch [17]: 0.09677357091330119
epoch [18]: 0.09247126185031397
epoch [19]: 0.09242277168509945
epoch [20]: 0.09101553422970765
